In [1]:
using Plots

In [2]:
using WriteVTK

In [3]:
push!(LOAD_PATH,"./")
using Bspline2

In [4]:
using FastGaussQuadrature

In [5]:
using BenchmarkTools

In [6]:
#リファインメント前
kh1=[0.0,0.0,1.0,1.0]
kh2=[0.0,0.0,1.0,1.0]
kh3=[0.0,0.0,1.0,1.0]

p11=1
p22=1
p33=1

u11=2
u22=2
u33=2
#リファインメント後
p1=3
p2=2
p3=1

n1=8
n2=1
n3=1

k1=vcat([0.0 for i in 1:p1+1],[i/n1 for i in 1:n1 ],[1.0 for i in 1:p1])
k2=vcat([0.0 for i in 1:p2+1],[i/n2 for i in 1:n2 ],[1.0 for i in 1:p2])
k3=vcat([0.0 for i in 1:p3+1],[i/n3 for i in 1:n3 ],[1.0 for i in 1:p3])

u1=n1+p1
u2=n2+p2
u3=n3+p3

N=u1*u2*u3

66

In [7]:
t1=110
t2=15
t3=10
t=collect(range(1.0e-10,stop=400,length=t1))
s1=collect(range(1.0e-10,stop=1.0-1.0e-10,length=t1)) 
s2=collect(range(1.0e-10,stop=1.0-1.0e-10,length=t2)) 
s3=collect(range(1.0e-10,stop=1.0-1.0e-10,length=t3))

10-element Array{Float64,1}:
 1.0e-10
 0.11111111118888889
 0.22222222227777777
 0.33333333336666665
 0.44444444445555553
 0.5555555555444445
 0.6666666666333333
 0.7777777777222222
 0.888888888811111
 0.9999999999

In [8]:
#紙の長さ、幅、厚さ、紙の端と端の間の距離を定義する。
Length=30  #長さ
Width=4  #幅
Thick=0.005  #厚さ:https://www.kamitsuuhan.jp/paper/thickness.php参照
gap=9  #端と端の間の距離
#これが初期状態の制御点になる。

9

In [9]:
#リファインメントと制御点の関係性
#新たな制御点を求めたい。khからkに移す。
#Cをまずは求める。
#資料のκをUの数列とする。
#リファインメント後の基底関数にκの値を入れた行列。
U1=[0.0 for i in 1:u1] #パラメータ空間のt1方向の何番目か。
U2=[0.0 for i in 1:u2]#パラメータ空間のt2方向の何番目か。
U3=[0.0 for i in 1:u3]
for i=1:u1
    U1[i]=k1[i]*(u1-i)/(u1+1)+k1[i+p1+1]*(i+1)/(u1+1)
end
for i=1:u2
    U2[i]=k2[i]*(u2-i)/(u2+1)+k2[i+p2+1]*(i+1)/(u2+1)
end
for i=1:u3
    U3[i]=k3[i]*(u3-i)/(u3+1)+k3[i+p3+1]*(i+1)/(u3+1)
end
#リファインメント後の基底関数に数値を入れた行列の定義終了。
C1=zeros(Float64,(u11,u1)) #t1方向の資料にあるC行列
C2=zeros(Float64,(u22,u2)) #t2方向の資料にあるC行列
C3=zeros(Float64,(u33,u3)) #t3方向の資料にあるC行列
Br1=zeros(Float64,(u1,u1)) #リファインメント後の基底関数の行列(t1方向)
Br2=zeros(Float64,(u2,u2)) #リファインメント後の基底関数の行列(t2方向)
Br3=zeros(Float64,(u3,u3)) #リファインメント後の基底関数の行列(t3方向)
B1=zeros(Float64,(u1,u11)) #リファインメント前の基底関数の行列(t1方向)
B2=zeros(Float64,(u2,u22)) #リファインメント前の基底関数の行列(t2方向)
B3=zeros(Float64,(u3,u33)) #リファインメント前の基底関数の行列(t3方向)
#上のBr1からB2に数値を入れていく。
for i=1:u1
    for j=1:u1
        Br1[i,j]=Bs(j,p1,k1,U1[i])
    end
end
for i=1:u2
    for j=1:u2
        Br2[i,j]=Bs(j,p2,k2,U2[i])
    end
end
for i=1:u3
    for j=1:u3
        Br3[i,j]=Bs(j,p3,k3,U3[i])
    end
end
#この上3つの行列の逆行列と下3つの行列をかける。
for i=1:u1
    for j=1:u11
        B1[i,j]=Bs(j,p11,kh1,U1[i])
    end
end
for i=1:u2
    for j=1:u22
        B2[i,j]=Bs(j,p22,kh2,U2[i])
    end
end
for i=1:u3
    for j=1:u33
        B3[i,j]=Bs(j,p33,kh3,U3[i])
    end
end
#数値代入終了。
#Br行列の逆行列をとりCを求める。
using LinearAlgebra
#Cが求まった。
det(Br1)
C1=Br1\B1
C2=Br2\B2
C3=Br3\B3
#制御点を求める。
#新たな制御点の行列をA1とA2と定め、前者がt1方向に制御点を動かす。後者がt2方向に動かす。
#以下では
#紙の長さ方向をx
#紙の幅をy
#紙の厚さをz
#と定義する。
at1=[0 : Length] #元のｘ方向の制御点
at2=[0 : Width] #元のｙ方向の制御点
at3=[0 : Thick] #元のz方向の制御点
At1=[0.0 for i in 1:u1] #新たなｘ方向の制御点
At2=[0.0 for i in 1:u2] #新たなｙ方向の制御点
At3=[0.0 for i in 1:u3] #新たなz方向の制御点
for i=1:u1
    At1[i]=C1[i,1]*0.0+C1[i,2]*Length
end
for i=1:u2
    At2[i]=C2[i,1]*0.0+C2[i,2]*Width
end
for i=1:u3
    At3[i]=C3[i,1]*0.0+C3[i,2]*Thick
end
C1
Br1
inv(Br1)
At1 #x方向の制御点
At2 #y方向の制御点
At3 #z方向の制御点
#制御点のリファインメントができた。
#次にその制御点の位置を正確に決める。
a=zeros(Float64,(3,N))
for j=1:u1
    for i=1:u2
        for k=1:u3
            a[1,((j-1)*u2+i-1)*u3+k]=At1[j]
            a[2,((j-1)*u2+i-1)*u3+k]=At2[i]
            a[3,((j-1)*u2+i-1)*u3+k]=At3[k]
        end
    end
end
a[2,:]

66-element Array{Float64,1}:
 0.0
 0.0
 2.0
 2.0
 4.0
 4.0
 0.0
 0.0
 2.0
 2.0
 4.0
 4.0
 0.0
 ⋮
 0.0
 0.0
 2.0
 2.0
 4.0
 4.0
 0.0
 0.0
 2.0
 2.0
 4.0
 4.0

In [10]:
#まずは点を用いて立方体を作る。
v1=zeros(Float64,(u1,t1))
for i=1:u1
    for j=1:t1
        v1[i,j]=Bs(i,p1,k1,s1[j]) #このa2のところに変数xを入れれば関数になる。
    end
end
v2=zeros(Float64,(u2,t2))
for i=1:u2
    for j=1:t2
        v2[i,j]=Bs(i,p2,k2,s2[j])
    end
end
v3=zeros(Float64,(u3,t3))
for i=1:u3
    for j=1:t3
        v3[i,j]=Bs(i,p3,k3,s3[j])
    end
end
R=zeros(Float64,(N,t1,t2,t3))
for i=1:N
    iI=Int(floor(Int(floor((i-1)/u3))/u2))+1
    iJ=Int(floor((i-1)/u3))%u2+1
    iK=(i-1)%u3+1
    for j=1:t1
        for k=1:t2
            for l=1:t3
                R[i,j,k,l]=v3[iK,l]*v2[iJ,k]*v1[iI,j]
            end
        end
    end
end

#制御点を用いて移動させる。
x=zeros(Float64,(t1,t2,t3))
y=zeros(Float64,(t1,t2,t3))
z=zeros(Float64,(t1,t2,t3))
for j=1:t1
    for k=1:t2
        for l=1:t3
            for i=1:N
                x[j,k,l]=x[j,k,l]+R[i,j,k,l]*a[1,i]
                y[j,k,l]=y[j,k,l]+R[i,j,k,l]*a[2,i]
                z[j,k,l]=z[j,k,l]+R[i,j,k,l]*a[3,i]        
            end
        end
    end
end
#問題の長方形が完成した。  
#この場所を一つの座標系で表示できるようにしないといけない。つまり、xの中の変数を一つにしなければならない。

In [11]:
x1=[0.0 for i in 1:t1*t2*t3]
y1=[0.0 for i in 1:t1*t2*t3]
z1=[0.0 for i in 1:t1*t2*t3]
for i=1:t1
    for j=1:t2
        for l=1:t3
        x1[((i-1)*t2+j-1)*t3+l]=x[i,j,l]
        y1[((i-1)*t2+j-1)*t3+l]=y[i,j,l]
        z1[((i-1)*t2+j-1)*t3+l]=z[i,j,l]
        end
    end
end
#これで実際のサイズに変換することができた。
#長方形ができた。
vtkfile=vtk_grid("tamesi",x1,y1,z1)
outfiles=vtk_save(vtkfile)

1-element Array{String,1}:
 "tamesi.vtr"

In [12]:
δ=zeros(Float64,(3,3))
for i=1:3
    for j=1:3
        if i==j
            δ[i,j]=1.0
        else
            continue
        end
    end
end
#ラメ定数
λ=40
μ=30
nip=7
#弾性定数
C=zeros(Float64,(3,3,3,3))
for i=1:3
    for j=1:3
        for k=1:3
            for l=1:3
                C[i,j,k,l]=λ*δ[i,j]*δ[k,l]+μ*δ[i,k]*δ[j,l]+μ*δ[i,l]*δ[j,k]
            end
        end
    end
end
#パラメーター空間と物理空間の間の関係。
#x=Length*t1
#y=Width*t2
#z=Thick*t3
#すべて1変数になっているので、物理空間からパラメーター空間への変換は

J=zeros(Float64,3)
J[1]=Length
J[2]=Width
J[3]=Thick

0.005

In [13]:
#非線形解を求めるのに必要な数値
function detR(U1,U2,U3,P1,P2,P3,K1,K2,K3,nip1,J1)::Array{Float64,5}
    DETr=zeros(Float64,(U1*U2*U3,3,U1*nip1,U2*nip1,U3*nip1))
    nodes,weights=gausslegendre(nip1)
    nodes1=zeros(Float64,U1*nip1)
    nodes2=zeros(Float64,U2*nip1)
    nodes3=zeros(Float64,U3*nip1)
    for i=1:U1
        for j=1:nip1
            nodes1[(i-1)*nip1+j]=nodes[j]*(K1[i+1]-K1[i])/2.0 .+(K1[i+1]+K1[i])/2.0
        end
    end
    for i=1:U2
        for j=1:nip1
            nodes2[(i-1)*nip1+j]=nodes[j]*(K2[i+1]-K2[i])/2.0 .+(K2[i+1]+K2[i])/2.0
        end
    end
    for i=1:U3
        for j=1:nip1
            nodes3[(i-1)*nip1+j]=nodes[j]*(K3[i+1]-K3[i])/2.0 .+(K3[i+1]+K3[i])/2.0
        end
    end
    
    for α=1:U1*U2*U3
        iI=Int(floor(Int(floor((α-1)/U3))/U2))+1 #t1方向で何番目か
        jI=Int(floor((α-1)/U3))%U2+1 #t2方向で何番目か
        kI=(α-1)%U3+1#t3方向で何番目か
        for i=1:U1*nip1
            for j=1:U2*nip1
                for l=1:U3*nip1
                    DETr[α,1,i,j,l]=Ḃs(iI,P1,K1,nodes1[i])*Bs(jI,P2,K2,nodes2[j])*Bs(kI,P3,K3,nodes3[l])/J1[1]
                    DETr[α,2,i,j,l]=Bs(iI,P1,K1,nodes1[i])*Ḃs(jI,P2,K2,nodes2[j])*Bs(kI,P3,K3,nodes3[l])/J1[2]
                    DETr[α,3,i,j,l]=Bs(iI,P1,K1,nodes1[i])*Bs(jI,P2,K2,nodes2[j])*Ḃs(kI,P3,K3,nodes3[l])/J1[3]
                end
            end
        end
    end
    return DETr
end

detR (generic function with 1 method)

In [14]:
#これは不変の値
DetR=zeros(Float64,(N,3,u1*nip,u2*nip,u3*nip))
@time DetR= detR(u1,u2,u3,p1,p2,p3,k1,k2,k3,nip,J)

  1.336833 seconds (1.74 M allocations: 125.080 MiB, 4.05% gc time)


66×3×77×21×14 Array{Float64,5}:
[:, :, 1, 1, 1] =
 -0.8  -0.5  -200.0
 -0.0  -0.0   200.0
 -0.0   0.5    -0.0
 -0.0   0.0     0.0
 -0.0   0.0    -0.0
 -0.0   0.0     0.0
  0.8  -0.0    -0.0
  0.0  -0.0     0.0
  0.0   0.0    -0.0
  0.0   0.0     0.0
  0.0   0.0    -0.0
  0.0   0.0     0.0
  0.0  -0.0    -0.0
  ⋮          
  0.0  -0.0    -0.0
  0.0  -0.0     0.0
  0.0   0.0    -0.0
  0.0   0.0     0.0
  0.0   0.0    -0.0
  0.0   0.0     0.0
  0.0  -0.0    -0.0
  0.0  -0.0     0.0
  0.0   0.0    -0.0
  0.0   0.0     0.0
  0.0   0.0    -0.0
  0.0   0.0     0.0

[:, :, 2, 1, 1] =
 -0.8  -0.5  -200.0
 -0.0  -0.0   200.0
 -0.0   0.5    -0.0
 -0.0   0.0     0.0
 -0.0   0.0    -0.0
 -0.0   0.0     0.0
  0.8  -0.0    -0.0
  0.0  -0.0     0.0
  0.0   0.0    -0.0
  0.0   0.0     0.0
  0.0   0.0    -0.0
  0.0   0.0     0.0
  0.0  -0.0    -0.0
  ⋮          
  0.0  -0.0    -0.0
  0.0  -0.0     0.0
  0.0   0.0    -0.0
  0.0   0.0     0.0
  0.0   0.0    -0.0
  0.0   0.0     0.0
  0.0  -0.0    -0.0
  0

In [15]:
using FastGaussQuadrature

In [16]:
LinerK=zeros(Float64,(3,3,N,N))
nodes,weights=gausslegendre(nip)
nodes1=zeros(Float64,nip)
nodes2=zeros(Float64,nip)
nodes3=zeros(Float64,nip)
@time for i=1:3
    for m=1:3
        for I=1:N
            iI=Int(floor(Int(floor((I-1)/u3))/u2))+1 #t1方向で何番目か
            jI=Int(floor((I-1)/u3))%u2+1 #t2方向で何番目か
            kI=(I-1)%u3+1
            for M=1:N
                iM=Int(floor(Int(floor((M-1)/u3))/u2))+1 #t1方向で何番目か
                jM=Int(floor((M-1)/u3))%u2+1 #t2方向で何番目か
                kM=(M-1)%u3+1
                if iI>=iM
                    maxI=iI
                    minI=iM
                else
                    maxI=iM
                    minI=iI
                end
                if jI>=jM
                    maxJ=jI
                    minJ=jM
                else
                    maxJ=jM
                    minJ=jI
                end
                if kI>=kM
                    maxK=kI
                    minK=kM
                else
                    maxK=kM
                    minK=kI
                end
                if maxI>minI+p1
                    continue
                elseif maxJ>minJ+p2
                    continue
                elseif maxK>minK+p3
                    continue
                else
                    for Ii=maxI:minI+p1
                        if k1[Ii+1]==k1[Ii]
                            continue
                        else
                        nodes1=nodes*(k1[Ii+1]-k1[Ii])/2.0 .+(k1[Ii+1]+k1[Ii])/2.0
                            for Jj=maxJ:minJ+p2
                                if k2[Jj+1]==k2[Jj]
                                    continue
                                else
                        nodes2=nodes*(k2[Jj+1]-k2[Jj])/2.0 .+(k2[Jj+1]+k2[Jj])/2.0
                                    for Kk=maxK:minK+p3
                                        if k3[Kk+1]==k3[Kk]
                                            continue
                                        else
                        nodes3=nodes*(k3[Kk+1]-k3[Kk])/2.0 .+(k3[Kk+1]+k3[Kk])/2.0
                                            for j=1:3
                                                for l=1:3
                                                    for a5=1:nip
                                                        for a6=1:nip
                                                            for a7=1:nip
                LinerK[i,m,I,M]=LinerK[i,m,I,M]+weights[a5]*weights[a6]*weights[a7]*C[i,j,m,l]*
                DetR[I,j,(Ii-1)*nip+a5,(Jj-1)*nip+a6,(Kk-1)*nip+a7]*
                DetR[M,l,(Ii-1)*nip+a5,(Jj-1)*nip+a6,(Kk-1)*nip+a7]*
                (k1[Ii+1]-k1[Ii])/2.0*(k2[Jj+1]-k2[Jj])/2.0*(k3[Kk+1]-k3[Kk])/2.0*J[1]*J[2]*J[3]
                                                            end
                                                        end
                                                    end
                                                end
                                            end
                                        end
                                    end
                                end
                            end
                        end
                    end
                end
            end
        end
    end
end

399.879139 seconds (5.42 G allocations: 104.384 GiB, 2.93% gc time)


In [ ]:
NewLinerK=zeros(Float64,(3*N,3*N))
for i=1:3*N
    for j=1:3*N
        if i==j
            NewLinerK[i,j]=1.0
        else
            continue
        end
    end
end

for v1=u3*u2+1:N
    for v2=u3*u2+1:N
        NewLinerK[3*v1-2,3*v2-2]=LinerK[1,1,v1,v2]
        NewLinerK[3*v1-2,3*v2-1]=LinerK[1,2,v1,v2]
        NewLinerK[3*v1-2,3*v2]=LinerK[1,3,v1,v2]
        NewLinerK[3*v1-1,3*v2-2]=LinerK[2,1,v1,v2]
        NewLinerK[3*v1-1,3*v2-1]=LinerK[2,2,v1,v2]
        NewLinerK[3*v1-1,3*v2]=LinerK[2,3,v1,v2]
        NewLinerK[3*v1,3*v2-2]=LinerK[3,1,v1,v2]
        NewLinerK[3*v1,3*v2-1]=LinerK[3,2,v1,v2]
        NewLinerK[3*v1,3*v2]=LinerK[3,3,v1,v2]
    end
end

In [ ]:
NewLinerK

In [31]:
Identity=zeros(Float64,(3*N,3*N))
for i=1:3*N
    for j=1:3*N
        if i==j
            Identity[i,j]=1.0
        else
            continue
        end
    end
end
Identity

198×198 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [32]:
NewLinerK=NewLinerK\Identity

198×198 Array{Float64,2}:
  1.0   0.0   0.0   0.0   0.0   0.0   0.0  …       0.0            0.0
  0.0   1.0   0.0   0.0   0.0   0.0   0.0          0.0            0.0
  0.0   0.0   1.0   0.0   0.0   0.0   0.0          0.0            0.0
  0.0   0.0   0.0   1.0   0.0   0.0   0.0          0.0            0.0
  0.0   0.0   0.0   0.0   1.0   0.0   0.0          0.0            0.0
  0.0   0.0   0.0   0.0   0.0   1.0   0.0  …       0.0            0.0
  0.0   0.0   0.0   0.0   0.0   0.0   1.0          0.0            0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0          0.0            0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0          0.0            0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0          0.0            0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0  …       0.0            0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0          0.0            0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0          0.0            0.0
  ⋮                             ⋮          ⋱                 
  

In [33]:
#端っこにかける力
Force=zeros(Float64,3)
Force[1]=0.0
Force[2]=0.0
Force[3]=-10.0

-10.0

In [34]:
smallb=zeros(Float64,(3,N))
nodes,weights=gausslegendre(nip)
#Q(i,j,x,y,a,U11,U22,p1,k1,p2,k2)
#Nx(I,p,x,y,p1,k1,p2,k2)
for j=1:3
    for i=1:N
        nodes,weights=gausslegendre(nip)
        nodes2=[0.0 for i in 1:nip]
        nodes3=zeros(Float64,nip)
        iI=Int(floor(Int(floor((i-1)/u3))/u2))+1 #t1方向で何番目か
        jI=Int(floor((i-1)/u3))%u2+1 #t2方向で何番目か
        kI=(i-1)%u3+1 #t3方向で何番目か
        for a4=jI:jI+p2
            nodes2=nodes*(k2[a4+1]-k2[a4])/2.0 .+(k2[a4+1]+k2[a4])/2.0
            for a5=kI:kI+p3
            nodes3=nodes*(k3[a5+1]-k3[a5])/2.0 .+(k3[a5]+k3[a5+1])/2.0
                for l=1:nip
                    for m=1:nip
            smallb[j,i]=smallb[j,i]+weights[m]*weights[l]*
            Bs(iI,p1,k1,1.0)*Bs(jI,p2,k2,nodes2[l])*Bs(kI,p3,k3,nodes3[m])*
            Force[j]*(k2[a4+1]-k2[a4])/2.0*(k3[a5+1]-k3[a5])/2.0*J[2]*J[3]
                    end
                end
            end
        end
    end
end

In [35]:
sum(smallb[3,:])

-0.20000000000000007

In [41]:
SmallB=zeros(Float64,3*N)
for i=u2*u3+1:N
    SmallB[3*i-2]=smallb[1,i]
    SmallB[3*i-1]=smallb[2,i]
    SmallB[3*i]=smallb[3,i]
end
LinerDis=zeros(Float64,3*N)
LinerDis=NewLinerK*SmallB
LinerU=zeros(Float64,(t1,t2))

110×15 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0


In [42]:
LinerU1=zeros(Float64,N)
LinerU2=zeros(Float64,N)
LinerU3=zeros(Float64,N)
for i=1:N
    LinerU1[i]=LinerDis[3*i-2]
    LinerU2[i]=LinerDis[3*i-1]
    LinerU3[i]=LinerDis[3*i]
end
for i=1:N
    a[1,i]=a[1,i]+LinerU1[i]
    a[2,i]=a[2,i]+LinerU2[i]
    a[3,i]=a[3,i]+LinerU3[i]
end

In [43]:
x=zeros(Float64,(t1,t2,t3))
y=zeros(Float64,(t1,t2,t3))
z=zeros(Float64,(t1,t2,t3))
for j=1:t1
    for k=1:t2
        for l=1:t3
            for i=1:N
                x[j,k,l]=x[j,k,l]+R[i,j,k,l]*a[1,i]
                y[j,k,l]=y[j,k,l]+R[i,j,k,l]*a[2,i]
                z[j,k,l]=z[j,k,l]+R[i,j,k,l]*a[3,i]        
            end
        end
    end
end
x1=[0.0 for i in 1:t1*t2*t3]
y1=[0.0 for i in 1:t1*t2*t3]
z1=[0.0 for i in 1:t1*t2*t3]
for i=1:t1
    for j=1:t2
        for l=1:t3
        x1[((i-1)*t2+j-1)*t3+l]=x[i,j,l]
        y1[((i-1)*t2+j-1)*t3+l]=y[i,j,l]
        z1[((i-1)*t2+j-1)*t3+l]=z[i,j,l]
        end
    end
end

In [46]:
a

3×66 Array{Float64,2}:
 -1.79856e-16  -1.79856e-16  -1.79856e-16  …  70598.2        -70538.4
  0.0           0.0           2.0              3328.34        -3317.47
  0.0           0.005         0.0                 8.76495e8       8.76495e8

In [45]:
plot(x1[:],y1[:],z1[:],marker=:circle)

In [ ]:
function intQ(a1,U1::Int64,U2::Int64,U3::Int64,K1,K2,K3,P1,P2,P3,nip1,N1,J1)::Array{Float64,5} #制御点の方向　微分する方向　制御点の情報
    IntQ=zeros(Float64,(3,3,U1*nip1,U2*nip1,U3*nip1))
    nodes,weights=gausslegendre(nip1)
    nodes1=zeros(Float64,U1*nip1)
    nodes2=zeros(Float64,U2*nip1)
    nodes3=zeros(Float64,U3*nip1)
    for i=1:U1
        for j=1:nip1
            nodes1[(i-1)*nip1+j]=nodes[j]*(K1[i+1]-K1[i])/2.0 .+(K1[i+1]+K1[i])/2.0
        end
    end
    for i=1:U2
        for j=1:nip1
            nodes2[(i-1)*nip1+j]=nodes[j]*(K2[i+1]-K2[i])/2.0 .+(K2[i+1]+K2[i])/2.0
        end
    end
    for i=1:U3
        for j=1:nip1
            nodes3[(i-1)*nip1+j]=nodes[j]*(K3[i+1]-K3[i])/2.0 .+(K3[i+1]+K3[i])/2.0
        end
    end
    
    for i=1:3
        for j=1:3
            for k=1:U1*nip1
                for l=1:U2*nip1
                    for m=1:U3*nip1
                        for I=1:N1
                        iI=Int(floor(Int(floor((I-1)/U3))/U2))+1 #t1方向で何番目か
                        jI=Int(floor((I-1)/U3))%U2+1 #t2方向で何番目か
                        kI=(I-1)%U3+1
                        IntQ[i,j,k,l,m]=IntQ[i,j,k,l,m]+a1[i,I]*DR[I,j,k,l,m]/J1[j]
                    end
                end
            end
        end
    end
    return IntQ
end  
#非線形解を求めるのに必要な数値終了

In [40]:
INTQ=zeros(Float64,(3,3,nip*u1,nip*u2,nip*u3))
INTQ=intQ(a,u1,u2,u3,k1,k2,k3,p1,p2,p3,nip,N,J)

3×3×77×21×14 Array{Float64,5}:
[:, :, 1, 1, 1] =
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

[:, :, 2, 1, 1] =
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

[:, :, 3, 1, 1] =
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

...

[:, :, 75, 1, 1] =
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

[:, :, 76, 1, 1] =
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

[:, :, 77, 1, 1] =
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

[:, :, 1, 2, 1] =
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

[:, :, 2, 2, 1] =
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

[:, :, 3, 2, 1] =
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

...

[:, :, 75, 2, 1] =
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

[:, :, 76, 2, 1] =
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

[:, :, 77, 2, 1] =
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

[:, :, 1, 3, 1] =
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

[:, :, 2, 3, 1] =
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

[:, :, 3, 3, 1] =
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0

In [43]:
function IntAB(C1,nip1::Int64,U1,U2,U3,P1::Int64,P2::Int64,P3,K1,K2,K3,δ1,Q,J1,DR)::Array{Float64,2} #制御点の方向　何番目か　制御点
    N1=U1*U2*U3
    IAB=zeros(Float64,(3,N1))
    nodes,weights=gausslegendre(nip1)
    for i=1:3
        for I=1:N1
            iI=Int(floor(Int(floor((I-1)/U3))/U2))+1 #t1方向で何番目か
            jI=Int(floor((I-1)/U3))%U2+1 #t2方向で何番目か
            kI=(I-1)%U3+1
            for Ii=iI:iI+P1
                if K1[Ii]==K1[Ii+1]
                    continue
                else
                    for Ji=jI:jI+P2
                        if K2[Ji+1]==K2[Ji]
                            continue
                        else
                            for Ki=kI:kI+P3
                                if K3[Ki+1]==K3[Ki]
                                    continue
                                else
                                for p=1:3
                                    for q=1:3
                                        for r=1:3
                                            for s=1:3
                                                for a6=1:nip1
                                                    for a7=1:nip1
                                                        for a8=1:nip1
                    IAB[i,I]=IAB[i,I]+
                    0.5*C1[p,q,r,s]*weights[a6]*weights[a7]*weights[a8]*
                    DR[I,p,(Ii-1)*nip1+a6,(Ji-1)*nip1+a7,(Ki-1)*nip1+a8]*
                    Q[i,q,(Ii-1)*nip1+a6,(Ji-1)*nip1+a7,(Ki-1)*nip1+a8]*
                    (Q[1,r,(Ii-1)*nip1+a6,(Ji-1)*nip1+a7,(Ki-1)*nip1+a8]*
                    Q[1,s,(Ii-1)*nip1+a6,(Ji-1)*nip1+a7,(Ki-1)*nip1+a8]+
                    Q[2,r,(Ii-1)*nip1+a6,(Ji-1)*nip1+a7,(Ki-1)*nip1+a8]*
                    Q[2,s,(Ii-1)*nip1+a6,(Ji-1)*nip1+a7,(Ki-1)*nip1+a8]-δ1[r,s])*
                    (K1[Ii+1]-K1[Ii])/2.0*(K2[Ji+1]-K2[Ji])/2.0*(K3[Ki+1]-K3[Ki])/2.0*
                    J1[1]*J1[2]*J1[3]
                                                            end
                                                        end
                                                    end
                                                end
                                            end
                                        end
                                    end
                                end
                            end
                        end
                    end
                end
            end
        end
    end
    return IAB
end

IntAB (generic function with 1 method)

In [42]:
function H(C1,nip1::Int64,U1::Int64,U2::Int64,U3::Int64,P1::Int64,P2::Int64,P3::Int64,K1,K2,K3,δ1,N,Q,J1,DR)::Array{Float64,4}
    INTH=zeros(Float64,(3,3,N,N))
    nodes,weights=gausslegendre(nip1)
    nodest1=[0.0 for a3 in 1:nip1]
    nodest2=[0.0 for a4 in 1:nip1]
    nodest3=[0.0 for a5 in 1:nip1]
    for i=1:3
        for m=1:3
            for I=1:N
                for M=1:N
                    iI=Int(floor(Int(floor((I-1)/U3))/U2))+1 #t1方向で何番目か
                    jI=Int(floor((I-1)/U3))%U2+1 #t2方向で何番目か
                    kI=(I-1)%U3+1
                    iM=Int(floor(Int(floor((M-1)/U3))/U2))+1 #t1方向で何番目か
                    jM=Int(floor((M-1)/U3))%U2+1 #t2方向で何番目か
                    kM=(M-1)%U3+1
                    if iI>=iM
                        maxI=iI
                        minI=iM
                    elseif iM>iI 
                        maxI=iM
                        minI=iI
                    end
                    if jI>=jM
                        maxJ=jI
                        minJ=jM
                    elseif jM>jI
                        maxJ=jM
                        minJ=jI
                    end
                    if kI>=kM
                        maxK=kI
                        minK=kM
                    elseif kM>kI
                        maxK=kM
                        minK=kI
                    end
                    if maxI>minI+P1
                        continue
                    elseif maxJ>minJ+P2
                        continue
                    elseif maxK>minK+P3
                        continue
                    else
                        for Ii=maxI:minI+P1
                            if K1[Ii+1]==K1[Ii]
                                continue
                            else
                                for Jj=maxJ:minJ+P2
                                    if K2[Jj+1]==K2[Jj]
                                        continue
                                    else
                                        for Kk=maxK:minK+P3
                                            if K3[Kk+1]==K3[Kk]
                                                continue
                                            else
                                                for p=1:3
                                                    for q=1:3
                                                        for r=1:3
                                                            for s=1:3
                                                                for a3=1:nip1
                                                                    for a4=1:nip1
                                                                        for a5=1:nip1
                    INTH[i,m,I,M]=INTH[i,m,I,M]+
                    0.5*C1[p,q,r,s]*weights[a3]*weights[a4]*weights[a5]*
                    DR[I,p,(Ii-1)*nip1+a3,(Jj-1)*nip1+a4,(Kk-1)*nip1+a5]*
                    (δ1[i,m]*DR[M,q,(Ii-1)*nip1+a3,(Jj-1)*nip1+a4,(Kk-1)*nip1+a5]*
                    (Q[1,r,(Ii-1)*nip1+a3,(Jj-1)*nip1+a4,(Kk-1)*nip1+a5]*
                    Q[1,s,(Ii-1)*nip1+a3,(Jj-1)*nip1+a4,(Kk-1)*nip1+a5]+
                    Q[2,r,(Ii-1)*nip1+a3,(Jj-1)*nip1+a4,(Kk-1)*nip1+a5]*
                    Q[2,s,(Ii-1)*nip1+a3,(Jj-1)*nip1+a4,(Kk-1)*nip1+a5]-δ1[r,s])+
                    2.0*DR[M,r,(Ii-1)*nip1+a3,(Jj-1)*nip1+a4,(Kk-1)*nip1+a5]*
                    Q[i,q,(Ii-1)*nip1+a3,(Jj-1)*nip1+a4,(Kk-1)*nip1+a5]*
                    Q[m,s,(Ii-1)*nip1+a3,(Jj-1)*nip1+a4,(Kk-1)*nip1+a5])*
                    (K1[Ii+1]-K1[Ii])/2.0*(K2[Jj+1]-K2[Jj])/2.0*(K3[Kk+1]-K3[Kk])/2.0*J1[1]*J1[2]*J1[3]
                                                                        end
                                                                    end
                                                                end
                                                            end
                                                        end
                                                    end
                                                end
                                            end
                                        end
                                    end
                                end
                            end
                        end
                    end
                end
            end
        end
    end
    return INTH
end

H (generic function with 1 method)

In [44]:
IntF=zeros(Float64,(3,N))
IntF=IntAB(C,nip,u1,u2,u3,p1,p2,p3,k1,k2,k3,δ,INTQ,J,DetR)-smallb
ArrayH=zeros(Float64,(3,3,N,N))
ArrayH=H(C,nip,u1,u2,u3,p1,p2,p3,k1,k2,k3,δ,N,INTQ,J,DetR)  


In [ ]:
for BCN=1:200
    INTQ=zeros(Float64,(3,3,nip*u1,nip*u2,nip*u3))
    INTQ=intQ(a,u1,u2,u3,k1,k2,k3,p1,p2,p3,nip,N,J)
    IntF=zeros(Float64,(3,N))
    IntF=IntAB(C,nip,u1,u2,u3,p1,p2,p3,k1,k2,k3,δ,INTQ,J)
    ArrayH=zeros(Float64,(3,3,N,N))
    ArrayH=H(C,nip,u1,u2,u3,p1,p2,p3,k1,k2,k3,δ,N,INTQ,J)  
    
    Re_ArrayH=zeros(Float64,(3*N,3*N))
    for i=1:3*u2*u3
        for j=1:3*u2*u3
            if i==j
                Re_ArrayH[i,j]=1.0
            else
                continue
            end
        end
    end
    for v1=u2*u3+1:N
        for v2=u3*u2+1:N
            Re_ArrayH[3*v1-2,2*v2-2]=ArrayH[1,1,v1,v2]
            Re_ArrayH[3*v1-2,3*v2-1]=ArrayH[1,2,v1,v2]
            Re_ArrayH[3*v1-1,3*v2-2]=ArrayH[2,1,v1,v2]
            Re_ArrayH[3*v1-1,3*v2-1]=ArrayH[2,2,v1,v2]
            Re_ArrayH[3*v1,3*v2-2]=ArrayH[3,1,v1,v2]
            Re_ArrayH[3*v1,3*v2-1]=ArrayH[3,2,v1,v2]
            Re_ArrayH[3*v1,3*v2]=ArrayH[3,3,v1,v2]
            Re_ArrayH[3*v1-2,3*v2]=ArrayH[1,3,v1,v2]
            Re_ArrayH[3*v1-1,3*v2]=ArrayH[2,3,v1,v2]
        end
    end
    
    Re_IntF=zeros(Float64,3*N)
    for i=u3*u2+1:N
        Re_IntF[3*i-2]=IntF[1,i]
        Re_IntF[3*i-1]=IntF[2,i]
        Re_IntF[3*i]=IntF[3,i]
    end
    
    Identity=zeros(Float64,(3*N,3*N))
    for i=1:3*N
        for j=1:3*N
            if i==j
                Identity[i,j]=1.0
            else
                continue
            end
        end
    end
    
    Re_ArrayH=Re_ArrayH\Identity
    NotLinerDis=zeros(Float64,3*N)
    NotLinerDis=Re_ArrayH*Re_IntF
    NotLinerU1=zeros(Float64,N)
    NotLinerU2=zeros(Float64,N)
    NotLinerU3=zeros(Float64,N)
    for i=1:N
        NotLinerU1[i]=NotLinerDis[3*i-2]
        NotLinerU2[i]=NotLinerDis[3*i-1]
        NotLinerU3[i]=NotLinerDis[3*i]
    end
    A1=zeros(Float64,N)
    A2=zeros(Float64,N)
    A3=zeros(Float64,N)
    A1=a[1,:]
    A2=a[2,:]
    A3=a[3,:]
    A=zeros(Float64,(3,N))
    
    A[1,:]=A1
    A[2,:]=A2
    A[3,:]=A3

    a[1,:]=A1-NotLinerU1
    a[2,:]=A2-NotLinerU2
    a[3,:]=A3-NotLinerU3
    
    nolmA=zeros(Float64,3)
    nolmA1=zeros(Float64,3)
    for i=1:3
        for j=1:N
            nolmA[i]=nolmA[i]+(a[i,j]-A[i,j])^2
        end
    end
    for i=1:3
        nolmA1[i]=sqrt(nolmA[i])
    end
    nolmA2=0.0
    nolmA2=sum(nolmA1)
    if nolmA2<=0.001
        break
    else
        println(a)
        continue
    end
end

In [ ]:
#まず輪を作る？？
